<a href="https://colab.research.google.com/github/avivyossef29/mixing-mechs-nlp/blob/nitzan%2Fbinding-tests/XGLM_7_5b_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install core libs (No IPython upgrade)
!pip install -q bitsandbytes transformers accelerate ipywidgets peft
!pip install -q "protobuf<6.0.0" "tensorboard<2.20.0" sentencepiece tqdm tabulate

# Re-clone your repo
!git clone https://github.com/NitzanZacharia/mixing-mechs-nlp.git

fatal: destination path 'mixing-mechs-nlp' already exists and is not an empty directory.


In [2]:
import sys
import importlib
from types import ModuleType

# This "tricks" Python into thinking 'imp' still exists
if 'imp' not in sys.modules:
    imp = ModuleType('imp')
    imp.reload = importlib.reload
    sys.modules['imp'] = imp
    print("✅ 'imp' module shimmed successfully. Autoreload should now work.")

✅ 'imp' module shimmed successfully. Autoreload should now work.


In [3]:
!pip install -q pyvene

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.6 MB/s eta 0:00:00


In [4]:
!pip install -q nnsight

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 5.0 MB/s eta 0:00:00


In [5]:
!pip install -U traitlets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.7.1
    Uninstalling traitlets-5.7.1:
      Successfully uninstalled traitlets-5.7.1


In [3]:
# Setup (run once)
%load_ext autoreload
%autoreload 2

import os
import sys

# Move to repo and add to path
repo_dir = '/content/mixing-mechs-nlp'
os.chdir(repo_dir)
sys.path.append(repo_dir)
sys.path.append(os.path.join(repo_dir, "CausalAbstraction"))

# Your logging and imports
import logging
logging.getLogger("transformers.configuration_utils").setLevel(logging.ERROR)
import transformers
transformers.logging.set_verbosity_error()

import torch
from tqdm.notebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

from grammar.schemas import SCHEMA_BOXES
from grammar.task_to_causal_model import multi_order_multi_schema_task_to_lookbacks_generic_causal_model
from training import get_counterfactual_datasets, sample_answerable_question_template

print("🚀 Script initialized successfully!")

🚀 Script initialized successfully!


In [4]:
# Configuration
model_id = "facebook/xglm-7.5B"
schema = SCHEMA_BOXES
num_instances = 20  # Number of binding groups (n=20 as in paper)
num_samples = 100   # Number of test samples (use 3000 for full test)
cat_indices_to_query = [0]  # Query by Object
cat_to_query = 1            # Answer is Box


In [5]:
from google.colab import output
output.enable_custom_widget_manager()

In [6]:
from transformers import BitsAndBytesConfig
import torch

# 1. Configure 4-bit quantization (Essential for T4)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16  # GPU likes float16, not float32
)

print(f"[+] Loading model: {model_id} on GPU...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",              # "auto" tells it to use the GPU (CUDA:0)
    low_cpu_mem_usage=True,         # Prevents crashing system RAM
    attn_implementation="eager"
)

print(f"✅ Model loaded on: {model.device}")


[+] Loading model: facebook/xglm-7.5B on GPU...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.92M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/15.0G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/15.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

✅ Model loaded on: cuda:0


In [9]:
print(f"    Model has {model.config.num_layers} layers")

    Model has 32 layers


In [10]:
# Generate Test Dataset
print(f"[+] Generating test dataset with {num_instances} binding groups...")
causal_model = multi_order_multi_schema_task_to_lookbacks_generic_causal_model(
    [schema], num_instances, num_fillers_per_item=0, fillers=False
)
causal_models = {schema.name: causal_model}

train_ds, test_ds, _ = get_counterfactual_datasets(
    None,  # No pipeline for filtering
    [schema],
    num_samples=num_samples,
    num_instances=num_instances,
    cat_indices_to_query=cat_indices_to_query,
    answer_cat_id=cat_to_query,
    do_assert=False,
    do_filter=False,
    causal_models=causal_models,
    sample_an_answerable_question=sample_answerable_question_template,
)

train = train_ds[schema.name][schema.name]
print(f"    Generated {len(train)} samples")


[+] Generating test dataset with 20 binding groups...
    Generated 100 samples


In [32]:
import re

def format_xglm_binding_prompt(prompt: str) -> str:
    # Extract the core logical statement
    context = prompt.split(' Respond')[0]
    try:
        question_part = prompt.split("nothing else:")[1].split("Box Answer:")[0].strip()
    except IndexError:
        question_part = "Which box is the object in?"

    # Few-Shot with explicit Language Tags (en:)
    # This keeps XGLM grounded in English
    few_shot = (
        "en: The scarf is in Box X, the basketball is in Box Y. "
        "Question: Which box is the scarf in? Answer: Box X\n"
        "###\n"
        "en: The cookie is in Box J, the keyboard is in Box K. "
        "Question: Which box is the keyboard in? Answer: Box K\n"
        "###\n"
    )

    return f"{few_shot}en: {context} Question: {question_part} Answer: Box"

def naive_box_checker(response: str, expected: str) -> bool:
    """
    Checks if the expected letter appears as a standalone word.
    Does NOT handle the 'BoxH' (no space) edge case.
    """
    # Find all words (sequences of alphanumeric characters)
    words = re.findall(r'\w+', response.upper())

    # Check if the expected letter is in that list
    return expected.strip().upper() in words

def generate_response(text, max_new_tokens=5):
    # Always add the BOS token for XGLM
    inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True).to(model.device)

    # XGLM often generates "</s>" (token 2) to end a sentence
    stop_strings = ["\n", "###", "</s>"]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded_output = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

    # Clean up output
    clean_output = decoded_output
    for stop_word in stop_strings:
        if stop_word in clean_output:
            clean_output = clean_output.split(stop_word)[0]

    return clean_output.strip()

In [90]:

#73 so far
def format_xglm_binding_prompt3(raw_context: str, question_object: str) -> str:
    # 1. Start with the BOS token
    bos = "<s>"

    # 2. Stronger Few-Shot (Using 3-4 objects helps the model handle longer lists)
    few_shot = (
        "The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. "
        "Question: Which box is the ocarina in? Answer: Box V. ### "
        "The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. "
        "Question: Which box is the sextant in? Answer: Box X. ### "
    )

    # 3. Clean the raw_context
    # This removes the "Respond in one word..." part that is already inside the data
    # It keeps only the sentences describing where objects are.
    clean_context = raw_context.split("Respond in one word")[0].strip()

    # 4. Final Formatting
    # We ensure a space after the context and a clean question at the end.
    test_context = f"{clean_context} "
    question = f"Question: Which box is the {question_object} in? Answer: Box"

    return f"{bos}{few_shot}{test_context}{question}"


In [96]:
#also 73
def format_xglm_binding_prompt2(raw_context: str, question_object: str) -> str:
    # 1. REMOVE MANUAL BOS (The tokenizer handles this)
    # Removing the manual "<s>" prevents the double-start-token confusion.

    # 2. DIVERSE FEW-SHOTS (Small Adjustment)
    # Example 1: Queries the FIRST item (Ocarina) -> Teaches Primacy
    # Example 2: Queries the LAST item (Anchor) -> Teaches Recency
    # This balance helps the model cover the whole context window.
    few_shot = (
        "The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. "
        "Question: Which box is the ocarina in? Answer: Box V. ### "

        "The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. "
        "Question: Which box is the plectrum in? Answer: Box Z. ### "
    )

    # 3. Clean Context (Kept exactly as you had it)
    clean_context = raw_context.split("Respond in one word")[0].strip()

    # 4. Final Formatting
    # We add a newline before the Question to help separation.
    test_context = f"{clean_context} "
    question = f"Question: Which box is the {question_object} in? Answer: Box"

    return f"{few_shot}{test_context}{question}"

# Keep your generation function exactly the same
def generate_response(text, max_new_tokens=2):
    # Ensure add_special_tokens=True is ON (default)
    inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True).to(model.device)

    stop_strings = ["\n", "###", "</s>"]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded_output = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

    clean_output = decoded_output
    for stop_word in stop_strings:
        if stop_word in clean_output:
            clean_output = clean_output.split(stop_word)[0]

    return clean_output.strip()

In [94]:
# Test Single Sample (for debugging)
sample = train[0]
raw_input = sample["input"]["raw_input"]
queried_object = sample["input"].get("Object.0.Query", None)

print("Raw input")
print(raw_input)
print(f"\nQueried object: {queried_object}")

# Get expected answer
forward_res = causal_model.run_forward(sample["input"])
expected = forward_res.get("answer", "")
if isinstance(expected, dict):
    expected = max(expected, key=lambda k: expected[k])
print(f"Expected answer: {expected}")

# Format and generate
prompt = format_xglm_binding_prompt2(raw_input, queried_object)
print(f"\nFormatted prompt")
print(prompt)

response = generate_response(prompt)
print(f"\nModel response: '{response}'")
print(f"Correct (strict): {naive_box_checker(response, expected)}")
print(f"Correct (original): {schema.checker(response, expected)}")


Raw input
The flower is in Box O, the leaf is in Box A, the brick is in Box P, the tie is in Box V, the wire is in Box Y, the crown is in Box S, the computer is in Box W, the tea is in Box T, the ice is in Box L, the newspaper is in Box X, the wheel is in Box F, the medicine is in Box I, the game is in Box H, the machine is in Box Q, the coffee is in Box Z, the plane is in Box D, the creature is in Box M, the document is in Box C, the map is in Box E, and the rose is in Box G. Respond in one word, only the answer and nothing else: Which box is the creature in? Box Answer:

Queried object: creature
Expected answer: M

Formatted prompt
The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. Question: Which box is the ocarina in? Answer: Box V. ### The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. Question: Which box is the anchor in? Answer: Box M. ### The flower is in Box O, the leaf is in Box A, the brick is in Box P, the tie is in Box V, the wi

In [97]:
from tqdm import tqdm
# Run Full Evaluation
print(f"[+] Evaluating {len(train)} samples...")

correct = 0
total = 0
position_correct = {}
position_total = {}
results = []

for i in tqdm(range(len(train))):
    sample = train[i]
    raw_input = sample["input"]["raw_input"]
    queried_object = sample["input"].get("Object.0.Query", None)

    # Get expected answer
    forward_res = causal_model.run_forward(sample["input"])
    expected = forward_res.get("answer", "")
    if isinstance(expected, dict):
        expected = max(expected, key=lambda k: expected[k])

    # Get query position
    query_position = sample["input"].get("metadata", {}).get("src_positional_index", -1)

    # Format and generate
    prompt = format_xglm_binding_prompt2(raw_input, queried_object)
    response = generate_response(prompt)

    # Check correctness using STRICT checker
    is_correct = naive_box_checker(response, expected)
    if is_correct:
        correct += 1
    total += 1

    # Track per-position accuracy
    if query_position >= 0:
        if query_position not in position_correct:
            position_correct[query_position] = 0
            position_total[query_position] = 0
        if is_correct:
            position_correct[query_position] += 1
        position_total[query_position] += 1

    results.append({
        "expected": expected,
        "response": response,
        "correct": is_correct,
        "position": query_position,
        "queried_object": queried_object,
        "full_query": prompt,
    })

# Print results
print(f"\n{'='*50}")
print(f"RESULTS: XGLM-7.5B on Binding Task (n={num_instances})")
print(f"{'='*50}")
print(f"Overall Accuracy: {correct/total:.2%} ({correct}/{total})")

if position_total:
    print(f"\nPer-Position Accuracy:")
    for pos in sorted(position_total.keys()):
        pos_acc = position_correct[pos] / position_total[pos]
        print(f"  Position {pos}: {pos_acc:.2%} ({position_correct[pos]}/{position_total[pos]})")


[+] Evaluating 100 samples...


100%|██████████| 100/100 [00:39<00:00,  2.50it/s]


RESULTS: XGLM-7.5B on Binding Task (n=20)
Overall Accuracy: 73.00% (73/100)

Per-Position Accuracy:
  Position 0: 100.00% (7/7)
  Position 1: 71.43% (5/7)
  Position 2: 33.33% (2/6)
  Position 3: 0.00% (0/2)
  Position 4: 55.56% (5/9)
  Position 5: 33.33% (2/6)
  Position 6: 33.33% (1/3)
  Position 7: 66.67% (2/3)
  Position 8: 100.00% (5/5)
  Position 9: 100.00% (3/3)
  Position 10: 80.00% (8/10)
  Position 11: 100.00% (2/2)
  Position 12: 80.00% (4/5)
  Position 13: 83.33% (5/6)
  Position 15: 100.00% (1/1)
  Position 16: 70.00% (7/10)
  Position 17: 83.33% (5/6)
  Position 18: 100.00% (4/4)
  Position 19: 100.00% (5/5)


In [54]:
import pandas as pd

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Filter for only incorrect responses
correct_ans = df[df['correct'] == True]

print(f"Total successes: {len(correct_ans)}")
print("\n--- Examples of correct Answers ---")
# Display the first 20 failures
# We use .head(20) to see enough variety
display(correct_ans[['queried_object', 'expected', 'response', 'position']].head(20))
pd.set_option('display.max_colwidth', None)

failed = df[df['correct'] == False]

print(f"Total Failures: {len(failed)}")
print("\n--- Examples of incorrect Answers ---")
# Display the first 20 failures
# We use .head(20) to see enough variety
display(failed[['queried_object', 'expected', 'response', 'position','full_query']].head(20))

Total successes: 9

--- Examples of correct Answers ---


,queried_object,expected,response,position
4,camera,A,A,17
32,chemical,L,L,0
53,wheel,C,C,19
54,guitar,Y,Y,0
70,clock,Q,Q,0
75,cross,H,H,0
80,document,A,A,17
89,ice,O,O,0
98,watch,A,A,4


Total Failures: 91

--- Examples of incorrect Answers ---


,queried_object,expected,response,position,full_query
0,creature,M,A,16,Context: The sun is in Box A. The moon is in Box B. The star is in Box C. The comet is in Box D. The planet is in Box E. The asteroid is in Box F. The nebula is in Box G. The galaxy is in Box H. The universe is in Box I. The void is in Box J. The light is in Box K. The dark is in Box L. Question: Which box is the sun in? Answer: Box A###\nContext: The flower is in Box O. the leaf is in Box A. the brick is in Box P. the tie is in Box V. the wire is in Box Y. the crown is in Box S. the computer is in Box W. the tea is in Box T. the ice is in Box L. the newspaper is in Box X. the wheel is in Box F. the medicine is in Box I. the game is in Box H. the machine is in Box Q. the coffee is in Box Z. the plane is in Box D. the creature is in Box M. the document is in Box C. the map is in Box E.. the rose is in Box G.\nQuestion: Which box is the creature in?\nAnswer: Box
1,bomb,U,B,16,Context: The sun is in Box A. The moon is in Box B. The star is in Box C. The comet is in Box D. The planet is in Box E. The asteroid is in Box F. The nebula is in Box G. The galaxy is in Box H. The universe is in Box I. The void is in Box J. The light is in Box K. The dark is in Box L. Question: Which box is the sun in? Answer: Box A###\nContext: The document is in Box H. the apple is in Box T. the block is in Box G. the cup is in Box D. the book is in Box K. the rose is in Box L. the train is in Box P. the stone is in Box V. the branch is in Box F. the dress is in Box E. the milk is in Box W. the map is in Box X. the ice is in Box O. the medicine is in Box Z. the disk is in Box N. the guitar is in Box Q. the bomb is in Box U. the game is in Box R. the ball is in Box C.. the gift is in Box M.\nQuestion: Which box is the bomb in?\nAnswer: Box
2,tie,Q,V,1,Context: The sun is in Box A. The moon is in Box B. The star is in Box C. The comet is in Box D. The planet is in Box E. The asteroid is in Box F. The nebula is in Box G. The galaxy is in Box H. The universe is in Box I. The void is in Box J. The light is in Box K. The dark is in Box L. Question: Which box is the sun in? Answer: Box A###\nContext: The stone is in Box V. the tie is in Box Q. the boat is in Box Y. the brain is in Box D. the file is in Box C. the hat is in Box R. the train is in Box B. the magazine is in Box F. the string is in Box S. the rock is in Box J. the boot is in Box A. the television is in Box L. the camera is in Box G. the coat is in Box W. the suit is in Box O. the bag is in Box H. the phone is in Box E. the plant is in Box I. the pot is in Box Z.. the chemical is in Box N.\nQuestion: Which box is the tie in?\nAnswer: Box
3,game,Y,A,4,Context: The sun is in Box A. The moon is in Box B. The star is in Box C. The comet is in Box D. The planet is in Box E. The asteroid is in Box F. The nebula is in Box G. The galaxy is in Box H. The universe is in Box I. The void is in Box J. The light is in Box K. The dark is in Box L. Question: Which box is the sun in? Answer: Box A###\nContext: The stone is in Box C. the brick is in Box A. the magnet is in Box H. the flower is in Box J. the game is in Box Y. the television is in Box X. the book is in Box F. the milk is in Box Q. the fan is in Box M. the plane is in Box T. the fish is in Box N. the disk is in Box I. the painting is in Box D. the apple is in Box B. the cup is in Box L. the bell is in Box W. the file is in Box G. the branch is in Box R. the tea is in Box P.. the seed is in Box V.\nQuestion: Which box is the game in?\nAnswer: Box
5,suit,V,A,2,Context: The sun is in Box A. The moon is in Box B. The star is in Box C. The comet is in Box D. The planet is in Box E. The asteroid is in Box F. The nebula is in Box G. The galaxy is in Box H. The universe is in Box I. The void is in Box J. The light is in Box K. The dark is in Box L. Question: Which box is the sun in? Answer: Box A###\nContext: The pot is in Box O. the guitar is in Box A. the suit is in Box V. the wire is in

Total Failures: 27

--- Examples of incorrect Answers ---


,queried_object,expected,response,position,full_query
5,suit,V,A.,2,"<s>The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. Question: Which box is the ocarina in? Answer: Box V. ### The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. Question: Which box is the sextant in? Answer: Box X. ### The pot is in Box O, the guitar is in Box A, the suit is in Box V, the wire is in Box X, the wheel is in Box W, the boat is in Box E, the cross is in Box I, the watch is in Box T, the bell is in Box R, the glass is in Box B, the block is in Box F, the meat is in Box M, the rock is in Box S, the cup is in Box C, the dress is in Box K, the sheet is in Box Q, the magnet is in Box N, the rose is in Box J, the disk is in Box Y, and the map is in Box D. Question: Which box is the suit in? Answer: Box"
10,pot,D,Q.,1,"<s>The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. Question: Which box is the ocarina in? Answer: Box V. ### The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. Question: Which box is the sextant in? Answer: Box X. ### The plane is in Box Q, the pot is in Box D, the flower is in Box N, the shell is in Box I, the guitar is in Box L, the brain is in Box C, the coat is in Box H, the milk is in Box Z, the dress is in Box R, the radio is in Box M, the brick is in Box V, the bag is in Box E, the newspaper is in Box U, the drink is in Box P, the bus is in Box J, the apple is in Box G, the phone is in Box Y, the crown is in Box F, the fish is in Box W, and the camera is in Box T. Question: Which box is the pot in? Answer: Box"
14,book,Z,X.,1,"<s>The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. Question: Which box is the ocarina in? Answer: Box V. ### The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. Question: Which box is the sextant in? Answer: Box X. ### The ice is in Box Y, the book is in Box Z, the game is in Box K, the shell is in Box T, the bomb is in Box L, the train is in Box A, the map is in Box O, the cross is in Box I, the creature is in Box Q, the brick is in Box R, the chemical is in Box N, the radio is in Box H, the watch is in Box X, the ball is in Box V, the machine is in Box U, the bell is in Box F, the boot is in Box E, the meat is in Box S, the pipe is in Box B, and the fan is in Box P. Question: Which box is the book in? Answer: Box"
15,coffee,X,D.,5,"<s>The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. Question: Which box is the ocarina in? Answer: Box V. ### The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. Question: Which box is the sextant in? Answer: Box X. ### The engine is in Box M, the game is in Box R, the tie is in Box E, the painting is in Box V, the watch is in Box Q, the coffee is in Box X, the seed is in Box J, the apple is in Box K, the disk is in Box A, the radio is in Box P, the drug is in Box L, the pipe is in Box U, the meat is in Box F, the brain is in Box H, the plant is in Box I, the document is in Box G, the hat is in Box W, the book is in Box S, the tea is in Box D, and the machine is in Box N. Question: Which box is the coffee in? Answer: Box"
16,cup,V,X.,16,"<s>The ocarina is in Box V, the abacus is in Box Q, the flute is in Box L. Question: Which box is the ocarina in? Answer: Box V. ### The sextant is in Box X, the plectrum is in Box Z, the anchor is in Box M. Question: Which box is the sextant in? Answer: Box X. ### The file is in Box K, the coat is in Box A, the glass is in Box S, the bus is in Box T, the brick is in Box J, the bill is in Box I, the wheel is in Box L, the letter is in Box G, the pot is in Box R, the crown is in Box M, the ball is in Box Q, the ticket is in Box O, the egg is in Box H, the tie is in Box C, the ice is in Box W, the train is in Box P, the cup is in Box V, the fish is in Box N, the boot is in Box X, and the painting is in Box Y. Question: Which box is the cup in? Answer: Box"
17,tea,